**Importing the data**

In [ ]:
# imports
from IPython.display import clear_output
from PIL import Image
import os
import json
# clear
clear_output(wait=True)
# installing kaggle
!pip install -q kaggle
# user and key info
info = {"username":"jackbellamy","key":"3442898ba5e904ac3a48eb5b59398805"}
# opening file
json_object = json.dumps(info, indent=4)
with open("kaggle.json", "w") as outfile:
    outfile.write(json_object)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# copying data to content file of kaggle which can be found on the left hand side of the screen
if not (os.path.exists("/content/nfl-big-data-bowl-2022")):
  !kaggle competitions download -c nfl-big-data-bowl-2022
  !mkdir nfl-big-data-bowl-2022
  !unzip nfl-big-data-bowl-2022.zip -d nfl-big-data-bowl-2022

**Calling data files to variables**

In [20]:
import pandas as pd
# reading the data to variables
scouting = pd.read_csv("/content/nfl-big-data-bowl-2022/PFFScoutingData.csv")
games = pd.read_csv("/content/nfl-big-data-bowl-2022/games.csv")
players = pd.read_csv("/content/nfl-big-data-bowl-2022/players.csv")
plays = pd.read_csv("/content/nfl-big-data-bowl-2022/plays.csv")
tracking2018 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2018.csv")
tracking2019 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2019.csv")
tracking2020 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2020.csv")

**Data sizes**

In [21]:
data = [scouting,games,players,plays,tracking2018,tracking2019,tracking2020]
# loop
for name in data:
  print(name.shape)

(19979, 20)
(764, 7)
(2732, 7)
(19979, 25)
(12777351, 18)
(12170933, 18)
(11821701, 18)


This shows the dataset set to be massive therefore as thid is supposed to be a 6 week project probably best to go for quality over quantity.

**Viewing data**

trying to spot anything that needs cleaning

In [ ]:
# shows the first n rows of the data, can change this by changing the n value
# plays.head()


**Cleaning data**

The 'birthDate' and 'collegeName' columns of 'players.csv' have missing values

Need to convert 'height' column values from 'players.csv' from the format feet-inch or inch to feet to allow for calculations etc

The 'birthDate' column should be parsed as DateTime instead of object?

Need to split YYYY (Year) from YYYY-MM-DD date format to allow for calculations etc

Replace abbrevated words with keywords in general, for example position of type of kick

Remove null values from the dataset for calculations etc

**Cleaning scouting**

In [22]:
scouting.head()

,gameId,playId,snapDetail,snapTime,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,gunners,puntRushers,specialTeamsSafeties,vises,kickContactType
0,2018090600,37,NaN,NaN,NaN,3.85,D,R,R,NaN,NaN,NaN,NaN,NaN,8-0-2,NaN,NaN,PHI 23; PHI 27,NaN,NaN
1,2018090600,366,OK,0.84,2.12,4.46,N,C,C,C,R,PHI 57,NaN,PHI 54,NaN,PHI 18; PHI 29,NaN,NaN,ATL 83; ATL 27; ATL 34; ATL 21,CC
2,2018090600,658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHI 58,NaN,NaN
3,2018090600,677,NaN,NaN,NaN,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,NaN,NaN,ATL 17; ATL 22,NaN,NaN
4,2018090600,872,OK,0.84,2.00,4.35,N,C,L,NaN,NaN,NaN,NaN,NaN,NaN,PHI 18; PHI 29,ATL 85,ATL 37,ATL 83; ATL 34; ATL 21,BF


converting letters to names (this can be generalised)

In [16]:
# converting letters to names (this can be generalised)
# 'kickType': abbreviated
kick_types = ['D','F','K','O','P','Q','S','B','N','R','A']
# kick type full names
kick_type_names = ['Deep','Flat','Free','Obvious','Pooch','Squib','Surprise','Deep','Normal','Rugby','Aussie']
# new names
kick_type_name = []
# looping through
for i in scouting["kickType"]:
  # filtering out NaN values
  if i in kick_types:
    index = kick_types.index(i)
    kick_type_name.append(kick_type_names[index])
  else:
    kick_type_name.append(i)
scouting["kick_type_name"] = kick_type_name

In [17]:
# new look at data
scouting[['kickType','kick_type_name']].head()
# have left NaN values in at the moment

,kickType,kick_type_name
0,D,Deep
1,N,Normal
2,NaN,NaN
3,D,Deep
4,N,Normal


splitting 'missedTackler' to team and number (can be generalised for other variables)

In [18]:
# splitting 'missedTackler' to team and number (can be generalised for other variables)
missed_tackler_split_team = []
missed_tackler_split_numbers = []
# splitting the data
missed_tackler_split = scouting["missedTackler"].str.split("; ", expand = True)
# loop for each
for i in range(0,len(scouting)):
  # specific data
  missed_tackler_split.iloc[i]
  # number of NaN
  nan_num = missed_tackler_split.iloc[i].isna().sum()
  if 5-nan_num == 0:
    missed_tackler_split_team.append(scouting['missedTackler'][i])
    missed_tackler_split_numbers.append(scouting['missedTackler'][i])
  else:
    info = []
    for j in range(0,(5-nan_num)):
      missed_tackler_split_i = missed_tackler_split[j][i].split(" ")
      info.append(missed_tackler_split_i)
    # team name
    missed_tackler_split_team.append(info[0][0])
    # numbers
    nums = []
    for k in range(0,5-nan_num):
      num = info[k][1]
      nums.append(num)
    missed_tackler_split_numbers.append(nums)
# adding new columns
scouting["missed_tackler_team"] = missed_tackler_split_team
scouting["missed_tackler_numbers"] = missed_tackler_split_numbers

In [19]:
# new look at data
scouting[['missedTackler','missed_tackler_team','missed_tackler_numbers']].head()

,missedTackler,missed_tackler_team,missed_tackler_numbers
0,NaN,NaN,NaN
1,PHI 57,PHI,[57]
2,NaN,NaN,NaN
3,ATL 83,ATL,[83]
4,NaN,NaN,NaN


'kickoffReturnFormation' to numerical

In [32]:
# 'kickoffReturnFormation' to numberic
# splitting the numbers
formation_split = scouting["kickoffReturnFormation"].str.split("-", n = 2, expand = True)
# new column names
formation_split.columns = ['Front','Mid','Back']
# adding new columns
scouting["formation_front"] = pd.to_numeric(formation_split["Front"])
scouting["formation_mid"] = pd.to_numeric(formation_split["Mid"])
scouting["formation_back"] = pd.to_numeric(formation_split["Back"])

In [34]:
# new look at data
scouting[['kickoffReturnFormation','formation_front','formation_mid','formation_back']].head()

,kickoffReturnFormation,formation_front,formation_mid,formation_back
0,8-0-2,8.0,0.0,2.0
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,8-0-2,8.0,0.0,2.0
4,NaN,NaN,NaN,NaN


**Cleaning games**

In [35]:
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2018090600,2018,1,09/06/2018,20:20:00,PHI,ATL
1,2018090900,2018,1,09/09/2018,13:00:00,BAL,BUF
2,2018090901,2018,1,09/09/2018,13:00:00,CLE,PIT
3,2018090902,2018,1,09/09/2018,13:00:00,IND,CIN
4,2018090903,2018,1,09/09/2018,13:00:00,MIA,TEN


In [ ]:
# splitting gameDate
# splitting the numbers
game_date_split = games["gameDate"].str.split("/", n = 2, expand = True)
# new column names
game_date_split.columns = ['Day','Month','Year']
# adding new columns
games["game_date_day"] = pd.to_numeric(game_date_split["Day"])
games["game_date_month"] = pd.to_numeric(game_date_split["Month"])
games["game_date_year"] = pd.to_numeric(game_date_split["Year"])

In [ ]:
# new look at data
games[['gameDate','game_date_day','game_date_month','game_date_year']].head()

**Cleaning players**

In [8]:
players.head()

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen


changing height to numerical values

In [36]:
# changing height to numerical values
# splitting the heights
height_split = players["height"].str.split("-", n = 1, expand = True)
# new column names
height_split.columns = ['Feet', 'Inch']
# adding new columns
players["height_feet"] = pd.to_numeric(height_split["Feet"])
players["height_inch"] = pd.to_numeric(height_split["Inch"])

In [ ]:
# new look at data
players[['height','height_feet','height_inch']].head()

In [38]:
# still have 73 feet
# getting rid of NaN values
players["height_inch"] = players["height_inch"].fillna(0)
# replacing heights that did not have '-' at start
for i in players["height_feet"]:
    if i > 7:
        j = 0.0833333 * i
        players["height_feet"].replace({i : j}, inplace=True)        
# inches to numerical value
players["height_inch"] = 0.0833333 * players["height_inch"]

In [40]:
# adding feet and inches to get one value
height_numerical = players["height_feet"] + players["height_inch"]
# adding new numerical value column
players["height_numerical"] = pd.to_numeric(height_numerical)

In [41]:
# new look at data
players[['height','height_numerical']].head()

,height,height_numerical
0,6-1,6.083333
1,6-0,6.000000
2,6-0,6.000000
3,6-1,6.083333
4,6-1,6.083333


getting year from 'birthDate'

In [ ]:
# getting year from 'birthDate'

**Cleaning plays**

In [ ]:
plays.head()

**Cleaning tracking**

In [ ]:
tracking2018.head()
# do not forget there's 3 seasons

In [ ]:
seasons = [2018, 2019, 2020]


**Extracting specific data**

In [ ]:
# x and y positions of players
x = tracking2018['x']
y = tracking2018['y']

# getting specific play and player, x and y
nfl_id = 39470
display_name = "Justin Tucker"

x_nfl_id_name = tracking2018.loc[((tracking2018["nflId"] == nfl_id) & (tracking2018["displayName"] == display_name)), "x"]
y_nfl_id_name = tracking2018.loc[((tracking2018["nflId"] == nfl_id) & (tracking2018["displayName"] == display_name)), "y"]

x_nfl_id_name.head(1000)

**Plotting Height**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# setting up figure
plt.figure(figsize=(10, 6))
# plotting
ax = sns.distplot(players['height_numerical'], bins=12)
ax.set_title('Height Distribution');

**Drawing pitch**

In [ ]:
import matplotlib.patches as patches
from matplotlib.patches import Arc
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

# Change size of the figure
plt.rcParams['figure.figsize'] = [12, 8]
def drawPitch(width, height, color="w"):
    fig = plt.figure()
    ax = plt.axes(xlim=(-10, width + 30), ylim=(-15, height + 5))
    plt.axis('off')

    # Grass around pitch
    rect = patches.Rectangle((-10, -5), width + 40, height + 10, linewidth=1, facecolor='darkgreen', capstyle='round') # facecolor='#3f995b'
    ax.add_patch(rect)
    ###################

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), width + 20, height, ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    ###################

    # vertical lines - every 5 yards
    for i in range(21):
        plt.plot([10 + 5 * i, 10 + 5 * i], [0, height], c="w", lw=2)
    ###################
        
    # distance markers - every 10 yards
    for yards in range(10, width, 10):
        yards_text = yards if yards <= width / 2 else width - yards
        # top markers
        plt.text(10 + yards - 2, height - 7.5, yards_text, size=15, c="w", weight="bold")
        # botoom markers
        plt.text(10 + yards - 2, 7.5, yards_text, size=15, c="w", weight="bold", rotation=180)
    ###################

    # yards markers - every yard
    # bottom markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [1, 3], color="w", lw=2)

    # top markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [height - 1, height - 3], color="w", lw=2)

    # middle bottom markers
    y = (height - 18.5) / 2
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [y, y + 2], color="w", lw=2)

    # middle top markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [height - y, height - y - 2], color="w", lw=2)
    ###################

    # draw home end zone
    plt.text(2.5, (height - 15) / 2, "HOME", size=30, c="w", weight="bold", rotation=90)
    rect = plt.Rectangle((0, 0), 10, height, ec=color, fc="#0064dc", lw=2)
    ax.add_patch(rect)

    # draw away end zone    
    plt.text(111, (height - 15) / 2, "AWAY", size=30, c="w", weight="bold", rotation=-90)
    rect = plt.Rectangle((width + 10, 0), 10, height, ec=color, fc="#c80014", lw=2)
    ax.add_patch(rect)
    ###################
    
    # draw extra spot point
    # left
    y = (height - 3) / 2
    plt.plot([10 + 2, 10 + 2], [y, y + 3], c="w", lw=2)
    
    # right
    plt.plot([width + 10 - 2, width + 10 - 2], [y, y + 3], c="w", lw=2)
    ###################
    
    # draw goalpost
    goal_width = 6 # yards
    y = (height - goal_width) / 2
    # left
    plt.plot([0, 0], [y, y + goal_width], "-", c="y", lw=10, ms=20)
    # right
    plt.plot([width + 20, width + 20], [y, y + goal_width], "-", c="y", lw=10, ms=20)
    
    return fig, ax
fig, ax = drawPitch(100, 53.3)

**Plotting on field**

In [ ]:
#imports
import numpy as np
from matplotlib import pyplot as plt
# drawing pitch
fig, ax = drawPitch(100, 53.3)
# getting specific x and y
nfl_id = 39470
display_name = "Justin Tucker"

#x_nfl_id = tracking2018.loc[tracking2018["nflId"] == nfl_id, "x"]
#y_nfl_id = tracking2018.loc[tracking2018["nflId"] == nfl_id, "y"]

x_name = tracking2018.loc[(tracking2018["displayName"] == display_name), "x"]
y_name = tracking2018.loc[(tracking2018["displayName"] == display_name), "y"]

x_pos = x[0:105] 
y_pos = y[0:105]

# plotting
#ax.plot(x_pos, y_pos, ls='dotted', linewidth=3, color='red')
tracking2018.query('gameId == 2018123000 and playId == 36').groupby('team')\
.plot(x='x', y='y', ax=ax, style='.')
# showing
plt.show()

# thoughts on this: 
# 1. obvs when image is blown up it will look crap therefore need to think of something
# 2. would work well with showing pass and the direction etc
# 3. could use it to show simulations of the 'ball' moving in different plays in vid presentation

**Simulating on NFL field image**

In [ ]:
def extract_one_game(game_id, play_id, df):
    game = df[(df.gameId == game_id) & (df.playId == play_id)]
    home = {}
    away = {}
    balls = []
    
    players = game.sort_values(['frameId'], ascending=True).groupby('nflId')
    for id, dx in players:
        jerseyNumber = int(dx.jerseyNumber.iloc[0])
        if dx.team.iloc[0] == "home":
            home[jerseyNumber] = list(zip(dx.x.tolist(), dx.y.tolist()))
        elif dx.team.iloc[0] == "away":
            away[jerseyNumber] = list(zip(dx.x.tolist(), dx.y.tolist()))


    ball_df = game.sort_values(['frameId'], ascending=True) 
    ball_df = ball_df[ball_df.team == "football"]
    balls = list(zip(ball_df.x.tolist(), ball_df.y.tolist()))
    return home, away, balls

In [ ]:
from matplotlib import animation
from IPython.display import HTML
def animate_one_play(game_id, play_id, df):
    fig, ax = drawPitch(100, 53.3)
    
    home, away, balls = extract_one_game(game_id, play_id, df)

    team_left, = ax.plot([], [], 'o', markersize=20, markerfacecolor="r", markeredgewidth=2, markeredgecolor="white", zorder=7)
    team_right, = ax.plot([], [], 'o', markersize=20, markerfacecolor="b", markeredgewidth=2, markeredgecolor="white", zorder=7)
    ball, = ax.plot([], [], 'o', markersize=10, markerfacecolor="black", markeredgewidth=2, markeredgecolor="white", zorder=7)
    drawings = [team_left, team_right, ball]

    def init():
        team_left.set_data([], [])
        team_right.set_data([], [])
        ball.set_data([], [])
        return drawings

    def draw_teams(i):
        X = []
        Y = []
        for k, v in home.items():
            x, y = v[i]
            X.append(x)
            Y.append(y)
        team_left.set_data(X, Y)
        
        X = []
        Y = []
        for k, v in away.items():
            x, y = v[i]
            X.append(x)
            Y.append(y)
        team_right.set_data(X, Y)

    def animate(i):
        draw_teams(i)
        
        x, y = balls[i]
        ball.set_data([x, y])
        return drawings
    
    # !May take a while!
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=len(balls), interval=100, blit=True)

    return HTML(anim.to_html5_video())

In [ ]:
# game_id, play_id, df
animate_one_play(2018123000, 36, tracking2018)

**This code can do the simulation in VS Code without a problem**

In [ ]:
# # imports 
# import numpy as np
# import matplotlib.pyplot as plt
# # input data
# x_pos = x[0:105]
# y_pos = y[0:105]
# # animating plot
# fig, ax = plt.subplots()
# for i in range(len(x_pos)):
#     # plotting field image
#     plt.rcParams["figure.figsize"] = [5.33, 12] # used [5.33,12] as pitch has dimensions [53.3,120]
#     plt.rcParams["figure.autolayout"] = True
#     im = plt.imread("pitch.jpg")
#     # fig and ax plots
#     im = ax.imshow(im,extent=[0, 120, 0, 53.3]) # respect to size of the field [53.3,120]
#     # plotting
#     ax.plot(x_pos[:i], y_pos[:i], ls='dotted', linewidth=1, color='red')
#     # labels
#     plt.xlabel('Home Sideline')
#     plt.ylabel('Home Endzone')
#     # pausing
#     plt.pause(0.01) # the ten times a second combo
# # showing
# plt.show()

**1. Looking at field position**

In [ ]:
import statistics as stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

kick_types = ['D','F','K','O','P','Q','S','B','N','R','A']

plt.figure(figsize=(10, 6))
# loop
for kick_type in kick_types:
  kick_data = scouting.loc[(scouting["kickType"] == kick_type), "hangTime"]
  ax = sns.distplot(kick_data,hist=False,label=kick_type)
  ax.set_title('Hang Time Distribution');

plt.legend()
plt.show()